### Adding elevation data to trajectories from NED13 tiles

Creating the cypy2env:

`conda create -n cypyenv -c conda-forge python=3.7 psycopg2 rasterio pandas ipykernel scipy geopy seaborn`

Then, use pip to install matplotlib, fitparse, and gitpython.

In [ ]:
import os
import re
import sys
import glob
import time
import json
import pickle
import datetime
import psycopg2
import subprocess
import rasterio

import geopy
import numpy as np
import pandas as pd
import seaborn as sns

from psycopg2 import sql
from scipy import interpolate

import matplotlib as mpl
from matplotlib import pyplot as plt

In [ ]:
import rasterio.warp
import rasterio.windows
import rasterio.enums
import rasterio.merge
import geopy.distance

In [ ]:
from cycler import cycler
red, blue, green, purple, orange, yellow, brown, pink, gray  = sns.color_palette('Set1')
mpl.rcParams['axes.prop_cycle'] = cycler(color=[blue, orange, green, red, brown, gray])

In [ ]:
%matplotlib 
%load_ext autoreload
%autoreload 2

In [ ]:
sys.path.insert(0, '../../dbutils/')
import dbutils

sys.path.insert(0, '../')
import cypy2

colors = sns.color_palette()

root = '/home/keith/Downloads/export_7989839-1'
wahoo_example = '2326365683.fit.gz'
garmin_example = '2122584483.fit.gz'
garmin_indoor_example = '2324139976.fit.gz'

In [ ]:
user, host = 'keith', 'localhost'

### cypy2 database

In [ ]:
# bear creek - happy valley loop from December 2018
activity_id = '20181208223529'

conn = psycopg2.connect(user=user, host=host, dbname='cypy2')
a = cypy2.Activity.from_db(conn, activity_id, kind='processed')

In [ ]:
d = dbutils.execute_query(
    conn,
    'select ST_AsGeoJSON(ST_Simplify(geom, .0005)) from proc_records where activity_id = %s',
    activity_id)

In [ ]:
acoords = np.array(json.loads(d[0][0])['coordinates'])

In [ ]:
plt.plot(acoords[:, 0], acoords[:, 1])

### Find OSM roads that are traversed by a given trajectory

This database is created by `strava/osm_to_pgsql.sh`.

In [ ]:
conn = psycopg2.connect(user=user, host=host, dbname='osm_roads')

In [ ]:
# bounding box for the test activity's trajectory
acoords.min(axis=0), acoords.max(axis=0)

In [ ]:
# find the road closest to each point along the activity's trajectory
query = '''
    select osm_id, name, ST_Distance(geom, ST_SetSRID(ST_MakePoint(%s, %s), 4326)) dist 
    from roads where name is not null
    and ST_Intersects(ST_MakeEnvelope(-122.3, 37.86, -122.15, 37.923, 4326), geom)
    order by dist limit 1'''

names, ids = [], []
for point in acoords:
    d = pd.read_sql(query % tuple(point[:2]), conn)
    names.append(d.iloc[0]['name'])
    ids.append(d.iloc[0].osm_id)

In [ ]:
# plot the trajectory and the name of the closest road at each point
fig, ax = plt.subplots()
ax.plot(acoords[:, 0], acoords[:, 1])
for point, name in zip(acoords, names):
     ax.annotate(name.split(' ')[0], (point[0], point[1]))

In [ ]:
# get the geometries of all of the closest roads
# (code < 5130 corresponds to major/minor roads)
query = '''
    select 'id' id, ST_AsGeoJSON(ST_Collect(geom))
    from (select * from roads where osm_id in %s) temp
    WHERE name is not null and code < 5130'''
d = dbutils.execute_query(conn, query, (tuple(ids),))

In [ ]:
# plot all of the closest roads
geoms =  json.loads(d[0][1])
for geom in geoms['geometries']:
    coords = np.array(geom['coordinates'][0])
    plt.plot(coords[:, 0], coords[:, 1], marker='')
    
plt.scatter(acoords[:, 0], acoords[:, 1], color='gray')

### Database of Strava routes from GPX files

In [ ]:
conn = psycopg2.connect(user=user, host=host, dbname='routes')

In [ ]:
# populate the database
from cypy2.strava import strava_routes_to_pgsql
filenames = glob.glob('/home/keith/Downloads/export_7989839-1/routes/*.gpx')
strava_routes_to_pgsql.insert_routes(conn, 'routes', filenames)

In [ ]:
# metadata from tracks table
d = pd.read_sql('''
    select track_id, type, name, ST_AsGeoJSON(ST_Simplify(geom, .001)) geom from tracks 
    where name like \'%King%\'''', conn)
d

In [ ]:
# coords for the King Ridge loop from the track_points table
d = pd.read_sql('''
    select track_id, ele, ST_AsGeoJSON(geom) as geom 
    from track_points where track_id = 344 order by point_order''', conn)

d['lon'] = [json.loads(geom)['coordinates'][0] for geom in d.geom]
d['lat'] = [json.loads(geom)['coordinates'][1] for geom in d.geom]

d.drop('geom', axis=1, inplace=True)
strava_route = d
d.head()

In [ ]:
# coords from geom column
plt.scatter(d.lon, d.lat)

### Interpolating Strava routes by distance

In [ ]:
lat, lon = strava_route.lat.values, strava_route.lon.values

In [ ]:
# distance between adjacent coordinates
dists = [0]
for ind in np.arange(1, lat.shape[0]):
    dist = geopy.distance.distance(
        (lat[ind-1], lon[ind-1]), 
        (lat[ind], lon[ind]))
    dists.append(dist.meters)

strava_route['dist'] = np.cumsum(dists)

In [ ]:
plt.scatter(lat, lon, s=np.array(dists)**2/100)

In [ ]:
# the distances are all between 8m and 100m 
# (while there appears to be a hard cut-off at 100m,
# the 8m is not a hard cut-off and is likely due to the tolerance in the rdp algorithm)
_ = plt.hist(dists, bins=300)

In [ ]:
# interpolate lat/lon coords by distance (which is approximately by arc length)
f = interpolate.interp1d(strava_route.dist.values, strava_route[['lat', 'lon']].values, axis=0, kind='cubic')
latlon = f(np.arange(0, strava_route.dist.max(), 10))

In [ ]:
plt.scatter(lat, lon, s=30)
plt.plot(latlon[:, 0], latlon[:, 1])

In [ ]:
dists = []
for ind in np.arange(1, vals_int.shape[0]):
    dists.append(geopy.distance.distance(latlon[ind-1, :], latlon[ind, :]).meters)
dists = np.array(dists)

In [ ]:
plt.hist(dists, bins=np.arange(9, 11, .01))

In [ ]:
# these large jumps in the interpolated coordinates 
# are associated with tiny but sharp loops in the original track
# (due, in at least one case, to a waypoint being placed 
# on the wrong side of the road in the strava route builder)
np.argwhere(np.array(dists) > 15)

### Looking up elevations from NED13 tiles

In [ ]:
src = rasterio.open(source_path)

In [ ]:
src.index(-122.59, 36.92), ~src.transform * (-122.59, 36.92)

In [ ]:
px, py = src.index(*tam)

(
# lon/lat of the pixel's top left corner
src.transform * (py, px), 

# lon/lat of the pixel's bottom right corner
src.transform * (py + 1, px + 1)
)

In [ ]:
res = 9.2592592593e-05

# lon/lat of the point at which to interpolate the elevation
tam = np.array([-122.59685, 37.92301])
tam_east = np.array([-122.5777, 37.9290])
home = [-122.4293, 37.7604]

def interp_elevation(src, point, kind='linear', sz=1):
    
    if kind is None:
        sz = 0
    elif kind=='linear':
        sz = 1
    elif kind=='cubic':
        sz = max(2, sz)
    else:
        raise ValueError('Invalid value for kind')

    # row/column of the pixel containing the point
    row, col = src.index(*point)

    window = rasterio.windows.Window(col - sz, row - sz, 2*sz + 1, 2*sz + 1)
    z = src.read(window=window, masked=False)[0]

    if kind is None:
        z_int = z[0][0]
    else:
        # lon/lat of the top left corner of the pixel at row, col
        px_lon, px_lat = src.transform * (col, row)

        # coordinates of the centers of the window pixels
        # (note that lats are negative because interp2d requires
        # that coordinates be strictly increasing)
        x = np.arange(-sz, sz + 1) * res + px_lon + res/2
        y = -(px_lat - np.arange(-sz, sz + 1) * res) + res/2

        # note: do not flatten z before passing to interp2d!
        # for some reason, the interpolated values change (and are incorrect)
        f = interpolate.interp2d(x, y, z, kind=kind, bounds_error=True)
        z_int = f(point[0], -point[1])
    
    return z_int

In [ ]:
interp_elevation(src, (-122.4471, 37.7546), kind=None, sz=3)*3.2808

In [ ]:
steps = np.arange(0, res*10, res/3)[:, None]
points = np.hstack((steps*0 + tam[0], steps + tam[1]))

In [ ]:
# check that interpolation is working as we expect it to
plt.plot([interp_elevation(src, point, kind=None) for point in points])
plt.plot([interp_elevation(src, point, kind='linear') for point in points])

In [ ]:
source_path = '/home/keith/raster-data/NED13/n38w123/grdn38w123_13/w001001.adf'
# source_path = '/media/keith/USGS_Backup/USGS/NED13/n38w119/grdn38w119_13/w001001.adf'

strava_route['ned13_raw'] = None
strava_route['ned13_int'] = None
with rasterio.open(source_path) as src:
    for ind, row in strava_route.iterrows():
        point = (row.lon, row.lat)
        strava_route.at[ind, 'ned13_int'] = interp_elevation(src, point, sz=3)
        strava_route.at[ind, 'ned13_raw'] = [v for v in src.sample([point], indexes=src.indexes)][0]

### Compare our elevations to Strava's elevations)

In [ ]:
plt.plot(strava_route.ned13_int.values.astype(float))
plt.plot(np.convolve(strava_route.ned13_int.values.astype(float), np.ones(5)/5, 'same'))
plt.plot(strava_route.ele.values.astype(float))

In [ ]:
# the resolution in meters of the NED13 tiles near SF
x = rasterio.warp.transform('EPSG:4269', 'EPSG:3857', [-122.5 + res, -122.5], [37.7, 37.7 + res])
np.diff(x[0]), np.diff(x[1])

### Merging NED13 tiles 
For trajectories not contained within single tiles.


Aside: NED13 tiles are about 70 miles by 90 miles.

In [ ]:
glob.glob('/home/keith/raster-data/NED13/*')

In [ ]:
def tilepath(position):
    lat, lon = position
    lon = np.abs(lon)
    path = os.path.join('n%dw%d' % (lat, lon), 'grdn%dw%d_13' % (lat, lon))
    return path

In [ ]:
root = '/home/keith/raster-data/NED13/'
tiles = [(38, 122), (38, 123), (39, 123), (39, 124)]
paths = [tilepath(tile) for tile in tiles]

In [ ]:
datasets = [rasterio.open(os.path.join(root, path)) for path in paths]

In [ ]:
# bounds are from King Ridge route
mosaic, mosaic_transform = rasterio.merge.merge(
    datasets, 
    nodata=0,    
    bounds=(-123.31, 38.4, -122.94, 38.63))

In [ ]:
~mosaic_transform * (-123.31, 38.63)

In [ ]:
plt.imshow(mosaic[0].astype(float))

In [ ]:
# NED13 resolution in meters
res = 9.2592592593e-05
x, y = rasterio.warp.transform('EPSG:4326', 'EPSG:3857', [-122, -122 + res], [38, 38 + res])
[round(val, 2) for val in (x[1] - x[0], y[1] - y[0])]

In [ ]:
# NED13 tile size in miles
x, y = rasterio.warp.transform('EPSG:4326', 'EPSG:3857', [-122, -121], [38, 39])
[round(val/1000 * .62) for val in (x[1] - x[0], y[1] - y[0])]